In [1]:
using StatsBase
using PyPlot
using Distributions
using JLD2, FileIO

In [2]:
function interaction(gamma::Float64, grid_width::Int64, site_x::Int64, site_y::Int64)
    xi = (site_x - 1) % grid_width;
    xj = div((site_x - 1 - xi), grid_width);
    yi = (site_y - 1) % grid_width;
    yj = div((site_y - 1 - yi), grid_width);
    di = xi - yi;
    dj = xj - yj;
    return exp(-gamma * (di^2 + dj^2)) * (site_x != site_y);
end

interaction (generic function with 1 method)

In [3]:
function gibbs_sampler(gamma::Float64, beta::Float64, grid_width::Int64, num_states::Int64, num_samples::Int64)
    N = grid_width^2;
    state = ones(Int64, N);
    marginal_counts = zeros(Int64, N, num_states);
    true_marginals = ones(Float64, N, num_states) / (1.0 * num_states);
    marginal_errors = zeros(Float64, num_samples);
    run_time = zeros(Float64,num_samples);
    for iter = 1:num_samples
        run_time[iter] = @elapsed begin
        i = rand(1:N);
        energies = zeros(num_states);
        for t = 1:num_states
            for j = 1:N
                energies[t] += beta * interaction(gamma, grid_width, i, j) * (state[j]==t);
            end
        end
        sample_probs = exp.(energies) ./ sum(exp.(energies));
        state[i] = sample(1:num_states, Weights(sample_probs));
        end
        for j = 1:N
            marginal_counts[j, state[j]] += 1;
        end
        marginal_errors[iter] = mean([norm((marginal_counts[j, :] / (1.0*iter)) - true_marginals[j, :]) for j = 1:N]);
        end
    return marginal_errors, run_time;
end

gibbs_sampler (generic function with 1 method)

In [4]:
srand(333);
@time potts_gibbs, potts_gibbs_time = gibbs_sampler(1.5, 4.6, 20, 10, 1000000);
@save "potts_gibbs_final.jld2" potts_gibbs
@save "potts_gibbs_final_time.jld2" potts_gibbs_time

440.885617 seconds (3.21 G allocations: 266.033 GiB, 2.48% gc time)


In [5]:
function param_L(gamma::Float64, beta::Float64, grid_width::Int64)
    N = grid_width^2;
    L = 0.0;
    for i = 1:N
        acc = 0.0;
        for j = 1:N
            if (i != j)
                acc += beta * interaction(gamma, grid_width, i, j);
            end
        end
        L = max(L, acc);
    end
    return L;
end

param_L (generic function with 1 method)

In [6]:
function poisson_weights(gamma::Float64, beta::Float64, lambda::Float64, grid_width::Int64)
    N = grid_width^2;
    weights = zeros(N,N);
    sum_M = zeros(N);
    norm_rho = zeros(N,N);
    L = 0.0;
    for i = 1:N
        acc = 0.0;
        for j = 1:N
            if (i != j)
                weights[i,j] = beta * interaction(gamma, grid_width, i, j);
                acc += weights[i,j];
            end
        end
        sum_M[i] = acc;
        L = max(L, acc);
    end
    Delta = sum_M * lambda / L + sum_M;
    rho = weights * lambda / L + weights;
    lm_L = weights * lambda / L;
    for k = 1:N
        norm_rho[k,:] = rho[k,:] ./ Delta[k];
    end
    return (L, Delta, rho, lm_L, norm_rho);
end

poisson_weights (generic function with 1 method)

In [7]:
function poisson_sampler(gamma::Float64, beta::Float64, lambda::Float64, grid_width::Int64, num_states::Int64, num_samples::Int64,L::Float64, Delta::Vector{Float64}, rho::Array{Float64,2}, lm_L::Array{Float64,2},norm_rho::Array{Float64,2})
    N = grid_width^2;
    state = ones(Int64, N);
    marginal_counts = zeros(Int64, N, num_states);
    true_marginals = ones(Float64, N, num_states) / (1.0 * num_states);
    marginal_errors = zeros(Float64, num_samples);
    run_time = zeros(Float64,num_samples);
    for iter = 1:num_samples
        run_time[iter] = @elapsed begin
        i = rand(1:N);
        B = rand(Poisson(Delta[i]));
        ind = find(norm_rho[i,:].>0);
        B_idx = sample(1:length(ind), Weights(norm_rho[i,ind]),B);
        idx = ind[B_idx];
        for b = 1:B
            if (state[idx[b]] != state[i])
                p = lm_L[i,idx[b]]./rho[i,idx[b]];
                if (rand()> p)
                    idx[b]=0;
                end
            end
        end        
        energies = zeros(num_states);
        for t = 1:num_states
            num=0;
            for j in idx
                if (j != 0)
                    energies[t] += log(1 + L / lambda) * (state[j]==t);
                    num=num+1;
                end
            end
        end
        
        sample_probs = exp.(energies) ./ sum(exp.(energies));
        state[i] = sample(1:num_states, Weights(sample_probs));
        end
        for j = 1:N
            marginal_counts[j, state[j]] += 1;
        end
        marginal_errors[iter] = mean([norm((marginal_counts[j, :] / (1.0*iter)) - true_marginals[j, :]) for j = 1:N]);
    end
    return marginal_errors, run_time;
end

poisson_sampler (generic function with 1 method)

In [17]:
srand(333);
lamb = 1 * param_L(1.5, 4.6, 20)^2
(L, Delta, rho, lm_L, norm_rho) = poisson_weights(1.5, 4.6, lamb, 20);
@time potts_poisson1, potts_poisson1_time = poisson_sampler(1.5, 4.6, lamb, 20, 10, 1000000, L, Delta, rho, lm_L, norm_rho);
@save "potts_pgibbs_new_final.jld2" potts_poisson1
@save "potts_pgibbs_new_final_time.jld2" potts_poisson1_time

178.688355 seconds (3.23 G allocations: 280.445 GiB, 6.67% gc time)


In [18]:
srand(333);
lamb = 0.1 * param_L(1.5, 4.6, 20)^2
(L, Delta, rho, lm_L, norm_rho) = poisson_weights(1.5, 4.6, lamb, 20);
@time potts_poisson01, potts_poisson01_time = poisson_sampler(1.5, 4.6, lamb, 20, 10, 1000000, L, Delta, rho, lm_L, norm_rho);
@save "potts_pgibbs01_new_final.jld2" potts_poisson01
@save "potts_pgibbs01_new_final_time.jld2" potts_poisson01_time

166.090342 seconds (3.23 G allocations: 280.056 GiB, 7.09% gc time)


In [19]:
srand(333);
lamb = 5 * param_L(1.5, 4.6, 20)^2
(L, Delta, rho, lm_L, norm_rho) = poisson_weights(1.5, 4.6, lamb, 20);
@time potts_poisson5, potts_poisson5_time = poisson_sampler(1.5, 4.6, lamb, 20, 10, 1000000, L, Delta, rho, lm_L, norm_rho);
@save "potts_pgibbs5_new_final.jld2" potts_poisson5
@save "potts_pgibbs5_new_final_time.jld2" potts_poisson5_time

197.877769 seconds (3.23 G allocations: 294.183 GiB, 6.33% gc time)
